### 데이터 불러오기

In [2]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

In [15]:
df = pd.read_csv('csv/mulit_classification_data.csv')
X_df = df


In [16]:
df

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [17]:
x = X_df.drop(["Pastry","Z_Scratch","K_Scatch","Stains","Dirtiness","Bumps","Other_Faults"], axis=1)
y = df[["Pastry","Z_Scratch","K_Scatch","Stains","Dirtiness","Bumps","Other_Faults"]].copy()

In [18]:
# 유저로 부터 입력을 받아 검증 데이터 셋을 사용할 것인지, 표준화를 사용할 것인지 정함.
print("[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요")
input_1 = input("[안내] 데이터를 표준화 하시겠습니까? : ")
# 표준화 진행 여부
if input_1 == 'y':
    scaler = StandardScaler()
    X = scaler.fit_transform(x)
    print("[안내] 데이터 표준화를 진행했습니다.")
else:
    X = x
    print("[안내] 데이터 표준화를 진행하지 않습니다.")
    
input_2 = input("[안내] 검증 데이터셋을 분리할까요? : ")



# 검증 데이터 진행 여부
if input_2 == 'y':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 추가로 분리했습니다.")

else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("[안내] 검증 데이터를 분리하지 않았습니다.")



[안내] 모델링을 시작합니다. (y or n)으로 진행해주세요
[안내] 데이터 표준화를 진행했습니다.
[안내] 검증 데이터를 추가로 분리했습니다.


### 모델 구현 
- 임시로 구현

In [28]:
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

# 모델구성
model = Sequential()
model.add(Dense(input_dim, input_dim=input_dim, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim*2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))
model.add(Dense(input_dim//2, activation='elu'))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dense(output_dim, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(patience=30, monitor='val_loss')

model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

print("validation loss, validation accuracy")
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

           

Epoch 1/1000
39/39 [==============================] - 4s 11ms/step - loss: 1.7955 - accuracy: 0.3465 - val_loss: 1.2917 - val_accuracy: 0.5707
Epoch 2/1000
39/39 [==============================] - 0s 4ms/step - loss: 1.4223 - accuracy: 0.4843 - val_loss: 1.1436 - val_accuracy: 0.6247
Epoch 3/1000
39/39 [==============================] - 0s 4ms/step - loss: 1.3388 - accuracy: 0.5077 - val_loss: 1.0617 - val_accuracy: 0.6504
Epoch 4/1000
39/39 [==============================] - 0s 4ms/step - loss: 1.2485 - accuracy: 0.5592 - val_loss: 0.9725 - val_accuracy: 0.6658
Epoch 5/1000
39/39 [==============================] - 0s 5ms/step - loss: 1.1796 - accuracy: 0.5745 - val_loss: 0.9269 - val_accuracy: 0.6735
Epoch 6/1000
39/39 [==============================] - 0s 5ms/step - loss: 1.1084 - accuracy: 0.6124 - val_loss: 0.8920 - val_accuracy: 0.6787
Epoch 7/1000
39/39 [==============================] - 0s 6ms/step - loss: 1.1136 - accuracy: 0.5955 - val_loss: 0.8517 - val_accuracy: 0.6992
Epoch

In [31]:
output_dim

7

In [35]:
# 복습한 내용으로 추가적인 진행
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt
# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}

# 모델 구현
for func in act_func:
    for i in lr_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(output_dim, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[early_stopping])
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


Epoch 1/1000
5/5 [==============================] - 2s 57ms/step - loss: 1.3138 - accuracy: 0.2869 - val_loss: 0.7882 - val_accuracy: 0.3625
Epoch 2/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.5766 - accuracy: 0.3175 - val_loss: 0.5433 - val_accuracy: 0.5141
Epoch 3/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.4644 - accuracy: 0.4811 - val_loss: 0.4746 - val_accuracy: 0.5296
Epoch 4/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.4073 - accuracy: 0.4722 - val_loss: 0.4820 - val_accuracy: 0.5167
Epoch 5/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.4507 - accuracy: 0.3932 - val_loss: 0.4964 - val_accuracy: 0.4807
Epoch 6/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.4484 - accuracy: 0.4359 - val_loss: 0.6398 - val_accuracy: 0.4499
Epoch 7/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.4248 - accuracy: 0.4867 - val_loss: 0.5559 - val_accuracy: 0.4242
Epoch 8/1000


In [34]:
# 복습한 내용으로 추가적인 진행
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt
# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}

# 모델 구현
for func in act_func:
    for i in lr_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(28, activation=func))
            model.add(Dense(14, activation=func))
            model.add(Dense(14, activation=func))
            model.add(Dense(output_dim, activation='softmax'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[early_stopping])
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


Epoch 1/1000
5/5 [==============================] - 2s 52ms/step - loss: 0.5119 - accuracy: 0.3247 - val_loss: 0.4557 - val_accuracy: 0.3753
Epoch 2/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.3429 - accuracy: 0.4335 - val_loss: 0.3901 - val_accuracy: 0.4344
Epoch 3/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2781 - accuracy: 0.5560 - val_loss: 0.3225 - val_accuracy: 0.5527
Epoch 4/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.2299 - accuracy: 0.6680 - val_loss: 0.2598 - val_accuracy: 0.6581
Epoch 5/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1994 - accuracy: 0.7123 - val_loss: 0.2568 - val_accuracy: 0.6452
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.1788 - accuracy: 0.7268 - val_loss: 0.2280 - val_accuracy: 0.6787
Epoch 7/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.1653 - accuracy: 0.7534 - val_loss: 0.2228 - val_accuracy: 0.6967
Epoch 8/1000


In [33]:
# 복습한 내용으로 추가적인 진행
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt
# dropout, 배치 정규화 추가
# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'softmax']
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.01, 0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}

# 모델 구현
for func in act_func:
    for i in lr_lst:
            model = Sequential()
            model.add(Dense(256, activation=func, input_dim=input_dim))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(28, activation=func))
            model.add(Dense(14, activation=func))
            model.add(Dense(14, activation=func))
            model.add(Dense(output_dim, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
            model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[early_stopping])
            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}


            
            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'dropout': 'yes', 'batch_normal': 'yes'}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print(best_time)
print(time_hyper)


Epoch 1/1000
5/5 [==============================] - 2s 70ms/step - loss: 1.4477 - accuracy: 0.2546 - val_loss: 0.7729 - val_accuracy: 0.4961
Epoch 2/1000
5/5 [==============================] - 0s 16ms/step - loss: 1.2515 - accuracy: 0.4674 - val_loss: 0.6679 - val_accuracy: 0.4165
Epoch 3/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.7760 - accuracy: 0.4899 - val_loss: 0.6961 - val_accuracy: 0.4576
Epoch 4/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.5834 - accuracy: 0.4996 - val_loss: 0.7385 - val_accuracy: 0.4242
Epoch 5/1000
5/5 [==============================] - 0s 14ms/step - loss: 0.5827 - accuracy: 0.5068 - val_loss: 0.7247 - val_accuracy: 0.4293
Epoch 6/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.5443 - accuracy: 0.5544 - val_loss: 0.6544 - val_accuracy: 0.5141
Epoch 7/1000
5/5 [==============================] - 0s 13ms/step - loss: 0.5557 - accuracy: 0.5624 - val_loss: 0.5951 - val_accuracy: 0.4730
Epoch 8/1000
